In [26]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, recall_score, roc_auc_score, accuracy_score, f1_score
from sklearn.model_selection import cross_val_score
from imblearn.pipeline import Pipeline, make_pipeline


In [27]:
X, y = make_classification(n_samples=12500, n_classes=2,n_features=10, weights=[0.99, 0.01], flip_y=0, random_state=2020)

## Splitting the dataset into the Training set and Test set


In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2020,stratify=y)

In [29]:
# Results from split
train_0, train_1 = len(y_train[y_train==0]), len(y_train[y_train==1])
test_0, test_1 = len(y_test[y_test==0]), len(y_test[y_test==1])
print('>Train: 0=%d, 1=%d, Test: 0=%d, 1=%d' % (train_0, train_1, test_0, test_1))


>Train: 0=9900, 1=100, Test: 0=2475, 1=25


## Creating CrossValidation Object

In [31]:
from sklearn.model_selection import StratifiedKFold 
kf = StratifiedKFold(n_splits=5, random_state=2020, shuffle=True)


## SMOTE Oversampling for the Minority Class

In [32]:
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
# summarize class distribution
print(Counter(y_train))
# define sample strategy
oversample = SMOTE(sampling_strategy=0.1)
undersample = RandomUnderSampler(sampling_strategy=1)

# fit and apply the transform
X_sample, y_sample = oversample.fit_resample(X_train, y_train)
print(Counter(y_sample))

# fit and apply the transform
X_sample, y_sample = undersample.fit_resample(X_sample, y_sample)
print(Counter(y_sample))


Counter({0: 9900, 1: 100})
Counter({0: 9900, 1: 990})
Counter({0: 990, 1: 990})


## Training the Logistic Regression model on the Training set

In [33]:
from sklearn.linear_model import LogisticRegression
lm = LogisticRegression(random_state = 2020)
lm.fit(X_sample, y_sample)

LogisticRegression(random_state=2020)

## Confusion Matrix

In [34]:
y_pred = lm.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[2330  145]
 [   3   22]]


In [38]:
#Warning: Do not sample the test set on cross validation
pipeline = make_pipeline(oversample, undersample, lm)
# Applying k-Fold Cross Validation
score = cross_val_score(pipeline, X_train, y_train, scoring='recall', cv=kf)
print('>Mean recall: %.1f, std: %.1f' % (score.mean()*100, score.std()*100))

>Mean recall: 87.0, std: 8.7


## What happens if we sample BEFORE Cross-validation?
Biased or over optimistic results

In [39]:
# This is a mistake
score = cross_val_score(lm, X_sample, y_sample, cv=kf, scoring='recall')
print('>Mean recall - WARNING: %.1f, std - WARNING: %.1f' % (score.mean()*100, score.std()*100))


>Mean recall - WARNING: 92.2, std - WARNING: 2.0


## Training the SVM model on the Training set


In [40]:
from sklearn.svm import SVC
svm = SVC(kernel = 'linear', random_state = 0)
svm.fit(X_sample, y_sample)

SVC(kernel='linear', random_state=0)

## Confusion Matrix

In [41]:
y_pred = svm.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[2323  152]
 [   4   21]]


In [42]:
#Warning: Do not sample the test set on cross validation
pipeline = make_pipeline(oversample, undersample, 
                              svm)
# Applying k-Fold Cross Validation
score = cross_val_score(pipeline, X_train, y_train, scoring='recall', cv=kf)
print('>Mean recall: %.1f, std: %.1f' % (score.mean()*100, score.std()*100))

>Mean recall: 88.0, std: 7.5


## Training the Random Forest model on the Training set


In [43]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 2020)
rf.fit(X_sample, y_sample)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=2020)

## Confusion Matrix

In [44]:
y_pred = rf.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[2411   64]
 [   5   20]]


In [45]:
#Warning: Do not sample the test set on cross validation
pipeline = make_pipeline(oversample, undersample, 
                              rf)
# Applying k-Fold Cross Validation
score = cross_val_score(pipeline, X_train, y_train, scoring='recall', cv=kf)
print('>Mean recall: %.1f, std: %.1f' % (score.mean()*100, score.std()*100))

>Mean recall: 86.0, std: 11.6


## Training the ANN model on the Training set

In [46]:
import keras
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

# Initialising the ANN
ann = Sequential()

# Adding the input layer and the first hidden layer
ann.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 10))

# Adding the second hidden layer
ann.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
ann.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
ann.fit(X_sample, y_sample, batch_size = 10, epochs = 10)



Using TensorFlow backend.


Epoch 1/10
1980/1980 [==============================] - 4s 2ms/step - loss: 0.6497 - accuracy: 0.7773
Epoch 2/10
1980/1980 [==============================] - 1s 385us/step - loss: 0.3225 - accuracy: 0.9268
Epoch 3/10
1980/1980 [==============================] - 1s 329us/step - loss: 0.1947 - accuracy: 0.9303
Epoch 4/10
1980/1980 [==============================] - 1s 350us/step - loss: 0.1820 - accuracy: 0.9263
Epoch 5/10
1980/1980 [==============================] - 1s 369us/step - loss: 0.1803 - accuracy: 0.9288
Epoch 6/10
1980/1980 [==============================] - 1s 457us/step - loss: 0.1801 - accuracy: 0.9247
Epoch 7/10
1980/1980 [==============================] - 1s 512us/step - loss: 0.1792 - accuracy: 0.9278
Epoch 8/10
1980/1980 [==============================] - 1s 445us/step - loss: 0.1791 - accuracy: 0.9283
Epoch 9/10
1980/1980 [==============================] - 1s 442us/step - loss: 0.1796 - accuracy: 0.9273
Epoch 10/10
1980/1980 [==============================] - 1s 395us/

## Get F1 score and Confusion Matrix

In [47]:
# Part 3 - Making the predictions and evaluating the model
# Predicting the Test set results
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[2319  156]
 [   4   21]]


In [48]:
#Applying Kfold CrossValidation
# Evaluating the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 10))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)

#Warning: Do not sample the test set on cross validation
pipeline = make_pipeline(oversample, undersample, 
                              classifier)

score = cross_val_score(pipeline, X = X_train, y = y_train, cv = kf, n_jobs = -1,scoring='recall')
print('>Mean recall: %.1f, std: %.1f' % (score.mean()*100, score.std()*100))

>Mean recall: 87.0, std: 8.7
